In [103]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import random
import os

train = pd.read_csv('C:/Users/tglee/Nananana/Deep_Learning/Final_Deep/DL_mini_project/2/train.csv')

In [104]:
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

In [105]:
Mytrain = train[train.columns[:-2]]

In [106]:
train.groupby(['subject'])['Activity'].value_counts()

subject  Activity          
1        WALKING               95
         STANDING              53
         WALKING_UPSTAIRS      53
         LAYING                50
         WALKING_DOWNSTAIRS    49
                               ..
30       WALKING               65
         WALKING_UPSTAIRS      65
         SITTING               62
         WALKING_DOWNSTAIRS    62
         STANDING              59
Name: count, Length: 126, dtype: int64

In [107]:
len(train['subject'].unique())

21

In [108]:
train['Activity'].value_counts()

Activity
LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: count, dtype: int64

In [109]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Mytrain_scaled = scaler.fit_transform(Mytrain)

In [110]:
y_train = train['Activity']

activity_mapping = {
    'WALKING': 0,
    'WALKING_UPSTAIRS': 1,
    'WALKING_DOWNSTAIRS': 2,
    'SITTING': 3,
    'STANDING': 4,
    'LAYING': 5
}


y_train = [activity_mapping[activity] for activity in y_train]

In [111]:
def build_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(6, activation='softmax')
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [112]:
k = 5
num_val = len(Mytrain) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)

    X_val = Mytrain[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate([Mytrain[:i * num_val], Mytrain[(i + 1) * num_val:]], axis=0)
    y_train_part = np.concatenate([y_train[:i * num_val], y_train[(i + 1) * num_val:]], axis=0)

    model = build_model()
    model.fit(X_train_part, y_train_part, epochs=num_epochs, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, np.array(y_val), verbose=0)
    all_scores.append(val_acc)


processing fold # 0


C:\Users\tglee\anaconda3\envs\tf-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.7189 - loss: 0.7203
Epoch 2/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9166 - loss: 0.2228
Epoch 3/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9485 - loss: 0.1397
Epoch 4/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9542 - loss: 0.1306
Epoch 5/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9658 - loss: 0.1030
Epoch 6/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.9643 - loss: 0.0982
Epoch 7/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9640 - loss: 0.0955
Epoch 8/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9708 - loss: 0.0738
Epoch 9/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9711 - loss: 0.0808
Epoch 10/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9641 - loss: 0.0955
Epoch 11/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9732 - loss: 0.0685
Epoch 12/100
184/184 ━━━━━━━━━

C:\Users\tglee\anaconda3\envs\tf-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.7455 - loss: 0.6974
Epoch 2/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9283 - loss: 0.1928
Epoch 3/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9448 - loss: 0.1521
Epoch 4/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9580 - loss: 0.1241
Epoch 5/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9615 - loss: 0.1041
Epoch 6/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9644 - loss: 0.0930
Epoch 7/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9679 - loss: 0.0892
Epoch 8/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9717 - loss: 0.0734
Epoch 9/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9737 - loss: 0.0730
Epoch 10/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9705 - loss: 0.0712
Epoch 11/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9730 - loss: 0.0716
Epoch 12/100
184/184 ━━━━━━━━

...

Epoch 1/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.7254 - loss: 0.7273
Epoch 2/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9235 - loss: 0.2031
Epoch 3/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9433 - loss: 0.1589
Epoch 4/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9517 - loss: 0.1247
Epoch 5/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9563 - loss: 0.1087
Epoch 6/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9655 - loss: 0.0986
Epoch 7/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.9691 - loss: 0.0851
Epoch 8/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9701 - loss: 0.0780
Epoch 9/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9708 - loss: 0.0763
Epoch 10/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9734 - loss: 0.0758
Epoch 11/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9758 - loss: 0.0663
Epoch 12/100
184/184 ━━━━━━━━

C:\Users\tglee\anaconda3\envs\tf-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.7223 - loss: 0.7445
Epoch 2/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9050 - loss: 0.2378
Epoch 3/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9352 - loss: 0.1675
Epoch 4/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9473 - loss: 0.1444
Epoch 5/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9469 - loss: 0.1319
Epoch 6/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9582 - loss: 0.1030
Epoch 7/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9620 - loss: 0.1003
Epoch 8/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9611 - loss: 0.0957
Epoch 9/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9618 - loss: 0.0946
Epoch 10/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9711 - loss: 0.0761
Epoch 11/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9725 - loss: 0.0685
Epoch 12/100
184/184 ━━━━━━━━

C:\Users\tglee\anaconda3\envs\tf-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.7124 - loss: 0.7383
Epoch 2/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9173 - loss: 0.2273
Epoch 3/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9391 - loss: 0.1778
Epoch 4/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9535 - loss: 0.1354
Epoch 5/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.9554 - loss: 0.1132
Epoch 6/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9546 - loss: 0.1182
Epoch 7/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9677 - loss: 0.0855
Epoch 8/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9697 - loss: 0.0803
Epoch 9/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9708 - loss: 0.0736
Epoch 10/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9697 - loss: 0.0752
Epoch 11/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9680 - loss: 0.0811
Epoch 12/100
184/184 ━━━━━━━

In [113]:
all_scores

[0.8408163189888,
 0.8891156315803528,
 0.8095238208770752,
 0.9272108674049377,
 0.9557822942733765]

In [114]:
mean_score = np.mean(all_scores)
mean_score

np.float64(0.8844897866249084)

### 결과1

In [116]:
# all_scores

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(6, activation='softmax')
# ])

In [117]:
# np.mean(all_scores)

### 결과2

In [119]:
# all_scores

#       tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')

In [120]:
# mean_score = np.mean(all_scores)
# mean_score

### 결과3

In [122]:
# all_scores

#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(6, activation='softmax')
#     ])

In [123]:

# mean_score = np.mean(all_scores)
# mean_score

### 결과 4

In [125]:
# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(250, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model

# all_scores
# 0.9727891087532043,
#  0.8680272102355957,
#  0.9401360750198364,
#  0.8557823300361633,
#  0.8802720904350281,
#  0.9278911352157593,
#  0.9768707752227783,
#  0.9428571462631226,
#  0.956462562084198,
#  0.9782313108444214]

# mean_score = np.mean(all_scores)
# 0.9299319744110107


### 결과 5

In [127]:
# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model
# all_scores

### 결과 6

In [129]:
  # model = tf.keras.Sequential([
  #     tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
  #     tf.keras.layers.Dense(300, activation='relu'),
  #     tf.keras.layers.Dense(300, activation='relu'),
  #     tf.keras.layers.Dense(6, activation='softmax')
  # ])
# all_scores

### 결과 7

In [131]:
  # model = tf.keras.Sequential([
  #     tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
  #     tf.keras.layers.Dense(300, activation='relu'),
  #     tf.keras.layers.Dense(6, activation='softmax')
  # ])
# all_scores

In [132]:
# mean_score = np.mean(all_scores)
# mean_score

### 최종

In [134]:
# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model

In [135]:
## 0.9206802666187286

In [136]:
# def build_model():
#   model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model

In [137]:
# 0.9323809623718262

In [138]:
# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model

In [139]:
# 0.9262585043907166

In [140]:
# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model
# all_scores

In [141]:
# 0.9261224508285523

In [142]:
# def build_model():
#   model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model

#epoch150

In [143]:
# 0.933605432510376